In [2]:
%load_ext google.cloud.bigquery

In [17]:
%%bigquery --project jchavezar-demo

WITH firstlasttouch AS (
    SELECT
        user_pseudo_id,
        MIN(event_timestamp) AS user_first_engagement,
        MAX(event_timestamp) AS user_last_engagement
    FROM 
        `firebase-public-project.analytics_153293282.events_*`
    WHERE 
        event_name='user_engagement'
    GROUP BY
        1
)
SELECT 
    user_pseudo_id,
    user_first_engagement,
    user_last_engagement,
    EXTRACT(MONTH from TIMESTAMP_MICROS(user_first_engagement)) as month,
    EXTRACT(DAYOFYEAR from TIMESTAMP_MICROS(user_first_engagement)) as julianday,
    EXTRACT(DAYOFWEEK from TIMESTAMP_MICROS(user_first_engagement)) as dayofweek,
    (user_first_engagement + 86400000000) AS ts_24hr_after_first_engagement,
    IF(user_last_engagement < (user_first_engagement + 86400000000),1,0) AS churned,
    IF(user_last_engagement < (user_first_engagement + 600000000),1,0) AS bounced
FROM 
    firstlasttouch

Downloading: 100%|██████████| 13588/13588 [00:01<00:00, 9254.75rows/s]


,user_pseudo_id,user_first_engagement,user_last_engagement,month,julianday,dayofweek,ts_24hr_after_first_engagement,churned,bounced
0,50D529BFD3EFEB1C8216DF9A321A552A,1535451530829009,1538609814876007,8,240,3,1535537930829009,0,0
1,137944D29FEB9F2C96187C5F6604F5DB,1532042482702018,1533178386830042,7,200,5,1532128882702018,0,0
2,8D86121A671969882C35F9E4C325EECE,1533892197036001,1538391613988044,8,222,6,1533978597036001,0,0
3,DA1F05090AB8762E28B7A27E0E4AFEE4,1535120174925006,1538402006318000,8,236,6,1535206574925006,0,0
4,67B3D12844D13C157D202B952EE9EA89,1535470505784011,1537382717683008,8,240,3,1535556905784011,0,0
...,...,...,...,...,...,...,...,...,...
13583,B6855B56FE0838DD2C78822AB08B3D02,1536769230810000,1536769445629002,9,255,4,1536855630810000,1,1
13584,512DAA6DE2804B4BB543D1BED5781189,1536736103606001,1536736252402000,9,255,4,1536822503606001,1,1
13585,C376FE9AC9D252C0445407FB97FA8CCD,1536761370590010,1536774089654017,9,255,4,1536847770590010,1,0
13586,86860125992009F7C7BC2944575FD219,1536744272944011,1536744783890001,9,255,4,1536830672944011,1,1


In [18]:
%%bigquery df

SELECT * FROM bqmlga4.returningusers


Downloading: 100%|██████████| 13588/13588 [00:01<00:00, 9257.20rows/s]


In [35]:
df['churned'].value_counts()

1    7440
0    6148
Name: churned, dtype: Int64

In [42]:
%%bigquery df_join

SELECT
    *
FROM
    bqmlga4.returningusers ret
LEFT OUTER JOIN
    bqmlga4.user_demographics dem
ON ret.user_pseudo_id = dem.user_pseudo_id
LEFT OUTER JOIN
    bqmlga4.user_aggregate_behavior beh
ON ret.user_pseudo_id = beh.user_pseudo_id
WHERE
    ret.bounced = 0

Downloading: 100%|██████████| 8031/8031 [00:01<00:00, 5678.76rows/s]


In [44]:
df_join['churned'].value_counts()/df_join['churned'].count()

0    0.765534
1    0.234466
Name: churned, dtype: Float64